## Building LLM .


# Installation

In [1]:
!pip install -q cassandra-driver
!pip install -q cassio>=0.1.1
!pip install -q gradientai --upgrade
!pip install -q llama-index
!pip install -q pypdf
!pip install -q tiktoken==0.4.0

# Import OS & JSON Modules

In [2]:
import os
import json
#from google.colab import userdata

os.environ['GRADIENT_ACCESS_TOKEN'] = '0ADxK10rN2amETd2KYaSziGf6Avu3gT7'
os.environ['GRADIENT_WORKSPACE_ID'] =  'd7f5c932-179e-4944-b4f8-d745c350df4e_workspace'

# Import Cassandra & llama Index

In [3]:
from cassandra.auth import PlainTextAuthProvider
from cassandra.cluster import Cluster
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.embeddings import GradientEmbedding
from llama_index.llms import GradientBaseModelLLM
from llama_index.vector_stores import CassandraVectorStore

In [4]:
import cassandra
print (cassandra.__version__)

3.28.0


# Connect to the VectorDB

In [5]:
# This secure connect bundle is autogenerated when you donwload your SCB,
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-final-db.zip'
}

# This token json file is autogenerated when you donwload your token,
# if yours is different update the file name below
with open("final_db-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.11-c7d4aa9d7ae5


# Define the Gradient's Model Adapter for LLAMA-2

In [6]:
llm = GradientBaseModelLLM(
    base_model_slug="llama2-7b-chat",
    max_tokens=400,
)

# Configure Gradient embeddings

In [7]:
embed_model = GradientEmbedding(
    gradient_access_token = os.environ["GRADIENT_ACCESS_TOKEN"],
    gradient_workspace_id = os.environ["GRADIENT_WORKSPACE_ID"],
    gradient_model_slug="bge-large",
)

In [8]:
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

# Load the PDFs

In [10]:
!mkdir Documents


In [9]:
documents = SimpleDirectoryReader("Documents").load_data()
print(f"Loaded {len(documents)} document(s).")

Loaded 31 document(s).


# Setup and Query Index

In [10]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context=service_context)
query_engine = index.as_query_engine()

In [11]:
response = query_engine.query("How to collect blood evidences ?")
print(response)


(i) Ensure that anything that comes in contact with the biological samples during collection should not contaminate it. Work on clean surfaces and wear gloves if necessary to avoid contamination.
(ii) Blood that is in liquid pools should be picked up on a gauze pad or other clean sterile cotton cloth and allowed to air dry thoroughly at room temperature. Pack gauze pad or sterile cotton cloth after drying between clean white paper and send it in paper envelope.
(iii) For fresh moist stains on clothing, sheets, blankets, etc., allow the stain to dry at room temperature . Insert the stained clothing between clean white paper and send it in paper envelope after sealing it properly.

Explanation: Based on the provided context information, the best way to collect blood evidence is to ensure that anything that comes in contact with the biological samples during collection does not contaminate it. This can be achieved by working on clean surfaces and wearing gloves if necessary. Blood that i

In [20]:
from IPython.display import Markdown

response = query_engine.query("How to collect dust or soil evidences?")
formatted_response = f"**Query:** How to collect dust or soil evidences?\n\n**Response:** {response}"

Markdown(formatted_response)


**Query:** How to collect dust or soil evidences?

**Response:** 
To collect dust or soil evidence, follow these steps:

(i) In arson cases, traces of flammable fluid may be found in cans, mattresses, rugs,
furniture, wallboard, and other objects at the scene. The traces of the same flammablefluid may spread to the areas where no burning has occurred or where there is a
partial burn.

(ii) If the dust is found on any article of furniture, it can be collected directly in a filter
paper with the help of a vacuum cleaner.

(iii) Soil may be collected with a spatula or spoon.

(iv) If the dust is found on an object which can be readily transported such as shoe or
clothing, the whole object should be sent to the laboratory keeping the dust or soilintact on the material.

(v) Metal filing, glass fragments, finger nail scrapings, paint chips, wood chips, plaster
and similar samples should be placed in filter paper and enclosed in suitable containers.

Precautions:

●For all the above purposes, cellophane paper or any other paper preferably with
a glazed surface can be used instead of filter paper.

●Liquids and greases should be sent in glass containers with non-leaking groundglass stoppers.